In [2]:
!pip install xgboost

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import xgboost as xgb

from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

## Data Cleaning

In [3]:
#data from kaggle dataset: "Prediction of music genre"
data1 = pd.read_csv("music_genre.csv")
#data from kaggle data set: "Spotify Tracks Dataset"
data2 = pd.read_csv("dataset.csv")

before_drop_na = len(data1)
data1 = data1.dropna()
after_drop_na = len(data1)
print("number of data dropped:", before_drop_na-after_drop_na)

#make key, mode into One Hot encoding
#music_genre into label encoding
onehot = OneHotEncoder(sparse=False)
onehot.fit(data1[['key', 'mode']])
onehotencoded = onehot.transform(data1[['key', 'mode']])
categorical_columns = [f'{col}_{cat}' for i, col in enumerate(data1[['key', 'mode']].columns) for cat in onehot.categories_[i]]
onehotdf = pd.DataFrame(onehotencoded, columns=categorical_columns)
data1 = pd.concat([data1.reset_index(), onehotdf.reset_index()], axis=1)

labelencoder = LabelEncoder()
data1['key_num'] = labelencoder.fit_transform(data1['key'])
data1['mode_num'] = labelencoder.fit_transform(data1['mode'])
data1['music_genre_num'] = labelencoder.fit_transform(data1['music_genre'])
data1

data1_edit = data1.drop(columns = ["index", "instance_id", "artist_name", "track_name", "music_genre", "key", "mode", "tempo", "obtained_date", "key_num", "mode_num", "music_genre_num"])
features = data1_edit.columns

# Separate features and target variable
X = data1_edit
y = data1['music_genre_num']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, data1["music_genre"], test_size=0.2, random_state=1)

y_train_encoded = labelencoder.fit_transform(y_train)
y_test_encoded = labelencoder.transform(y_test)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the CNN model
model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(2),
    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_encoded, epochs=5, batch_size=32, validation_data=(X_test, y_test_encoded))


<ipython-input-3-14def6aae7de>:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data2 = pd.read_csv("dataset.csv")
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


number of data dropped: 6
Epoch 1/5
888/888 [==============================] - 11s 11ms/step - loss: 1.3332 - accuracy: 0.5048 - val_loss: 1.1724 - val_accuracy: 0.5645
Epoch 2/5
888/888 [==============================] - 11s 12ms/step - loss: 1.1250 - accuracy: 0.5848 - val_loss: 1.1267 - val_accuracy: 0.5869
Epoch 3/5
888/888 [==============================] - 8s 8ms/step - loss: 1.0876 - accuracy: 0.5969 - val_loss: 1.0859 - val_accuracy: 0.5969
Epoch 4/5
888/888 [==============================] - 6s 6ms/step - loss: 1.0586 - accuracy: 0.6113 - val_loss: 1.0810 - val_accuracy: 0.6022
Epoch 5/5
888/888 [==============================] - 5s 5ms/step - loss: 1.0377 - accuracy: 0.6149 - val_loss: 1.0515 - val_accuracy: 0.6093


In [4]:
intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer('flatten').output)
cnn_features_train = intermediate_layer_model.predict(X_train)
cnn_features_test = intermediate_layer_model.predict(X_test)

222/222 [==============================] - 0s 2ms/step


## XGBoost and CNN Data 1 Model

In [5]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(cnn_features_train, y_train_encoded)

y_pred = xgb_model.predict(cnn_features_test)
accuracy = accuracy_score(y_test_encoded, y_pred)

print("Accuracy for Data 1:", accuracy)
print("Classification Report for Data 1:")
print(classification_report(y_test_encoded, y_pred))

Accuracy for Data 1: 0.6163167535578413
Classification Report for Data 1:
              precision    recall  f1-score   support

           0       0.46      0.51      0.48       962
           1       0.78      0.78      0.78      1033
           2       0.55      0.52      0.53       960
           3       0.55      0.61      0.58      1021
           4       0.64      0.61      0.63      1018
           5       0.57      0.54      0.55      1019
           6       0.76      0.81      0.79       977
           7       0.07      0.01      0.02       107

    accuracy                           0.62      7097
   macro avg       0.55      0.55      0.54      7097
weighted avg       0.61      0.62      0.61      7097



## Splitting dataset, Standardizing features, Training data etc.

In [6]:
data2 = data2.dropna()
#make track_genre into label encoding
labelencoder = LabelEncoder()
data2['track_genre_num'] = labelencoder.fit_transform(data2['track_genre'])
data2

features2 = ["popularity", "duration_ms", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature"]
len(features2)

# Separate features and target variable for data 2
X2 = data2[features2]  # Features for data 2
y2 = data2['track_genre_num']  # Target variable for data 2

# Split the dataset into training and testing sets for data 2
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=1)

y_train_encoded2 = labelencoder.fit_transform(y_train2)
y_test_encoded2 = labelencoder.transform(y_test2)

# Standardize the features for data 2
scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train2)
X_test2 = scaler.transform(X_test2)

# Reshape the data for CNN input for data 2
X_train2 = X_train2.reshape(X_train2.shape[0], X_train2.shape[1], 1)
X_test2 = X_test2.reshape(X_test2.shape[0], X_test2.shape[1], 1)

# Define the CNN model for data 2
model2 = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(X_train2.shape[1], 1)),
    Conv1D(64, 3, activation='relu'),
    Conv1D(128, 3, activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(np.unique(y2)), activation='softmax')
])

# Compile the model
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model for data 2
model2.fit(X_train2, y_train_encoded2, epochs=5, batch_size=32, validation_data=(X_test2, y_test_encoded2))


<ipython-input-6-0f54aaaf1f32>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['track_genre_num'] = labelencoder.fit_transform(data2['track_genre'])


Epoch 1/5
1805/1805 [==============================] - 14s 7ms/step - loss: 2.9948 - accuracy: 0.2188 - val_loss: 2.6442 - val_accuracy: 0.2838
Epoch 2/5
1805/1805 [==============================] - 13s 7ms/step - loss: 2.5497 - accuracy: 0.3010 - val_loss: 2.5011 - val_accuracy: 0.3072
Epoch 3/5
1805/1805 [==============================] - 13s 7ms/step - loss: 2.4118 - accuracy: 0.3290 - val_loss: 2.3940 - val_accuracy: 0.3325
Epoch 4/5
1805/1805 [==============================] - 13s 7ms/step - loss: 2.3173 - accuracy: 0.3467 - val_loss: 2.3433 - val_accuracy: 0.3457
Epoch 5/5
1805/1805 [==============================] - 13s 7ms/step - loss: 2.2400 - accuracy: 0.3650 - val_loss: 2.3318 - val_accuracy: 0.3458


In [7]:
intermediate_layer_model2 = Model(inputs=model2.input, outputs=model2.get_layer('flatten_1').output)
cnn_features_train2 = intermediate_layer_model2.predict(X_train2)
cnn_features_test2 = intermediate_layer_model2.predict(X_test2)


452/452 [==============================] - 1s 2ms/step


## Data 2 Model

In [ ]:
xgb_model = xgb.XGBClassifier(n_jobs=-1)
xgb_model.fit(cnn_features_train2, y_train_encoded2)

y_pred2 = xgb_model.predict(cnn_features_test2)
accuracy2 = accuracy_score(y_test_encoded2, y_pred2)

print("Accuracy for Data 2:", accuracy2)
print("Classification Report for Data 2:")
print(classification_report(y_test_encoded2, y_pred2))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3da032f1-e5ab-4726-ac09-eb3d9c053730' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>